# Run: rdrp1 Invertebrates

```
Lead     : ababaian
Issue    : 
Version  : v0.4.0-dev : diamond-dev branch
start    : 2021 01 13
complete : 2021 01 xx
files    : ~/serratus/notebook/210108_ab/
s3_files : s3://serratus-public/notebook/210108_ab/
output   : s3://serratus-public/out/210113_rInv/
```

### Intro/Objectives

> Everything that lives and dies is a hologram. Nothing is as real as that.


### Invertebrate

- File: `inv_SraRunInfo.csv`

Search Term: 
```
("Metazoa"[Organism] NOT "Vertebrata"[Organism]) ("type_rnaseq"[Filter] OR "metagenomic"[Filter] OR "metatranscriptomic"[Filter]) AND cluster_public[prop] AND "platform illumina"[Properties]
```

SRA Accessed: `2020/12/31`

Results: `184 133 / 184 729`

[See: 201230_sraRunInfo_Update.ipynb](./201230_sraRunInfo_Update.ipynb)


In [ ]:
## Fire up EC2 Instance
#sudo yum install -y docker
#sudo yum install -y git
#sudo service docker start

## Download latest serratus repo
#git clone -b diamond-dev https://github.com/ababaian/serratus.git; cd serratus/containers

## If you want to upload containers to your repository, include this.
#export DOCKERHUB_USER='serratusbio' # optional
#sudo docker login # optional
#
## Build all containers and upload them docker hub repo (if available)
#./build_containers.sh

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/210111_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/210111_ab'

# date and version
date
git rev-parse HEAD # commit version

Wed Jan 13 14:42:38 PST 2021
2ddf24166fdded0bc89a96b2541bcbc7442f98f9


In [2]:
cd $WORK

# Copy from pmito5 pilot test
ln -s ../201230_sra/inv_SraRunInfo.csv ./

md5sum *
ls -alh

ln: failed to create symbolic link './inv_SraRunInfo.csv': File exists
0cdc0f966e6fbdd1a35a63b3260806de  inv_SraRunInfo.csv
e0ada71c13d1d80776383d7c0120f58c  mamm_SraRunInfo.csv
cf36fb9216bc478da656f00b193569aa  vert_SraRunInfo.csv
21984f68344cee144268f9fc9ac703e8  viro_SraRunInfo.csv
total 16K
drwxrwxr-x  2 artem artem 4.0K Jan 13 14:32 .
drwxr-xr-x 44 artem artem  12K Jan 13 14:42 ..
lrwxrwxrwx  1 artem artem   32 Jan 13 14:32 inv_SraRunInfo.csv -> ../201230_sra/inv_SraRunInfo.csv
lrwxrwxrwx  1 artem artem   33 Jan 12 13:52 mamm_SraRunInfo.csv -> ../201230_sra/mamm_SraRunInfo.csv
lrwxrwxrwx  1 artem artem   33 Jan 11 13:42 vert_SraRunInfo.csv -> ../201230_sra/vert_SraRunInfo.csv
lrwxrwxrwx  1 artem artem   33 Jan 13 10:13 viro_SraRunInfo.csv -> ../201230_sra/viro_SraRunInfo.csv


In [ ]:
#aws s3 sync ./ $S3_WORK/

### Terraform Initialize

In [ ]:
## UPDATE serratus/terraform/main/main.tf
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

In [3]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..b854970 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,68 +89,73 @@ re

     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corres

module.monitoring.module.ecs_cluster.aws_iam_role_policy_attachment.instance_attachment: Creation complete after 1s [id=SerratusEcsInstanceRole-monitor-20210113224307400100000004]
module.align.module.iam_role.aws_iam_role_policy.cloudwatch: Creating...
module.monitoring.module.ecs_cluster.aws_iam_instance_profile.p: Creation complete after 1s [id=instance-profile-serratus-monitor]
module.align.module.iam_role.aws_iam_instance_profile.profile: Creating...
module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creating...
module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creating...
module.download.aws_iam_role_policy.AdjustAutoScaling: Creation complete after 1s [id=SerratusIamRole-serratus-dl:AdjustAutoScaling-serratus-dl]
module.merge.module.iam_role.aws_iam_instance_profile.profile: Creating...
module.align.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAc

module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-06effcbe2737ef12a]
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-00b7512866291359d]
module.download.aws_launch_configuration.worker: Creating...
local_file.create_tunnel: Creating...
local_file.create_tunnel: Creation complete after 0s [id=9c8d0a370fcbdac72109a7e823fc86ab65550e4c]
module.align.aws_launch_configuration.worker: Creating...
module.merge.aws_launch_configuration.worker: Creating...
local_file.hosts: Creating...
local_file.hosts: Creation complete after 0s [id=debbe9747d1b2f4c6fe5e7f8d125e613c319a8e1]
module.merge.aws_laun

Serratus Config backup 
```
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 25,
  "ALIGN_SCALING_CONSTANT": 0.0215,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 1000,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 550,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 10,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}
```

In [5]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>

channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
channel 4: open failed: connect failed: Connection refused
Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [6]:
cd $WORK
BATCH='inv_SraRunInfo.csv'
wc -l $WORK/$BATCH

184729 /home/artem/serratus/notebook/210111_ab/inv_SraRunInfo.csv


In [9]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/inv_SraRunInfo.csv
  date: Wed Jan 13 14:51:31 PST 2021
  wc  : 184729 /home/artem/serratus/notebook/210111_ab/inv_SraRunInfo.csv
  md5 : 0cdc0f966e6fbdd1a35a63b3260806de  /home/artem/serratus/notebook/210111_ab/inv_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
2c4c392f9bc12f10937912919c871e6f  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
5640daf930559628c374077c20ba2d52  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
fb1325f816e0164aeb11a09afcf60d1e  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
87425ed5c4457eebee68c3893b1d4ee6  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":10000,

## Run Serratus

In [25]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -sT serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 40,
  "ALIGN_SCALING_CONSTANT": 0.09,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 0,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 0,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 200,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":40,"ALIGN_SCALING_CONSTANT":0.09,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":0,"CLEAR_INTERVAL":999999,"DL_ARGS":"","DL_MAX_INCREASE":20,"DL_SCALING_CONSTANT":0.1,"DL_SCALING_ENABLE":true,"DL_SCALING_MAX":0,"GENOME":"rdrp1","MERGE_ARGS":"protein","MERGE_MAX_INCREASE":25,"MERGE_SCALING_CONSTANT":0.1,"MERGE_SCALING_ENABLE":true,"MERGE_SCALING_MAX":200,"SCALING_INT

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Regular Shutdown
cd $TF
terraform destroy

## Nuke Shutdown
#cd $TF
#
#aws ec2 describe-instances \
#  --filter Name=tag:Name,Values=serratus-align-instance \
#  > align_instances.json
#
#jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
#  | pv -l \
#  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

# Notes: 



psummary: `https://serratus-public.s3.amazonaws.com/out/210108_r1p/psummary/<SRA>.psummary`

SRA site: `https://www.ncbi.nlm.nih.gov/sra/?term=<SRA>`
